In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [51]:
url_inicial = 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/archive'
url_root = 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/archive'
r=requests.get(url_inicial)

In [52]:
soup = BeautifulSoup(r.text, 'html.parser')

In [53]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/116',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/115',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/114',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/113',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/112',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/111',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/110',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/109',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/107',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/96',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/74',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/70',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/48',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/31',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/30',
 'http://rmcg.ge

In [54]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('ul', class_='articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [55]:
vol2

['http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1690',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1673',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1641',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1617',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1636',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1651',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1649',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1627',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1607',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1592',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1573',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1560',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/927',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1054',
 'http://rmcg.geociencias.unam.mx/i

In [56]:
def get_url_items(sopa,url):
    box1 = soup1.find('ul', class_='articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [57]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/archive


In [58]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

140

In [59]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [60]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Ciencias Geológicas'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Geociencias'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item abstract').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [61]:
list_scraper=list_scraper[0:290]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [62]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Covers,,"AbstractCovers, Revista Mexicana de Ciencias G...",http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
1,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Covers,,"AbstractCovers, Revista Mexicana de Ciencias G...",http://rmcg.geociencias.unam.mx/index.php/rmcg...,None
2,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Spherical data validation of rock discontinuit...,,AbstractThis work presents a methodology for t...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
3,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Sewage sludge from Taxco de Alarcón wastewater...,,AbstractThe management and disposal of the sew...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
4,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Mobile mud layer underneath the desiccated maa...,,AbstractWe surveyed a subsurface layer of satu...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
...,...,...,...,...,...,...,...,...
135,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Estudio de la estructura cortical en la zona c...,,"AbstractWe conducted a gravity study, based on...",http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
136,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Identificación de fases portadoras y flujos de...,,AbstractIn this work the variations in the con...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
137,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,"CaK-clinoptilolite, KNa-chabazite, KNa-heuland...",,"AbstractThis paper presents the occurrence, mi...",http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
138,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Caracterización de las depresiones kársticas (...,,AbstractThe occurrence of floods in karst depr...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...


In [63]:
df_catalogo.to_csv('Revista15.01.csv', index=False)

In [35]:
df=df_catalogo.drop(df_catalogo.index[[38]])
df

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,"Geology of the Sierra of Charcas, State of San...",None,AbstractThe mapped area is 130 km north of the...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
1,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Early Cretaceous tectono-sedimentary evolution...,None,AbstractThe Bisbee basin is considered to be t...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
2,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Structural evolution of the Sierra de Juárez m...,None,"AbstractThe western side of Sierra de Juárez, ...",http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
3,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Origin of the magnetite boundary in the Penins...,None,AbstractIn the Peninsular Ranges of southern C...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
4,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,"Las Américas Formation, Tlalpujahua, Michocán,...",None,AbstractThe early Pliocene Las Américas Format...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
...,...,...,...,...,...,...,...,...
256,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Un paleolago del NW mexicano en su conjunto pa...,None,AbstractEl estudio de las oscilaciones climáti...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
257,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Observaciones generales sobre la estructura de...,None,AbstractA partir de I`étude de coupes levées s...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
258,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Mamíferos meocénicos de México: contribución a...,None,AbstractThe Tertiary mammalian fauna of Mexico...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
259,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Caracteres químicos principales del vulcanismo...,None,AbstractLes soixante analyses chimiques nouvel...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...


In [22]:
df.to_csv('Revista15.04.csv', index=False)